# Deformation of solids

**Prashant K. Jha**

*Department of Mechanical Engineering, South Dakota School of Mines and Technology, Rapid City, SD, USA*

In this tutorial, we model the deformation of elastic materials using fenics. The problems considered include:
1. Deformation of a linear elastic material modeled using small deformation theory. This results in a linear variational problem. I
2. Deformation of an orthotropic elastic material (as an example of anisotropic material). 
3. Deformation of a hyperelastic material assuming large deformation theory. The resulting partial differential equation is highly nonlinear in the displacement.

## Large deformation of linear elastic materials

Consider a 3-D beam $\Omega = \{(x,y,z): x\in (0, L),\; y\in (0,W),\; z\in (0,H)\}$ as shown in the figure. The beam is fixed on the left surface (at $x=0$), $\Gamma_l = \{(x,y,z) \in \Omega: x = 0\}$, and is subjected to external traction $\boldsymbol{t}$ on the right surface (at $x=L$), $\Gamma_r = \{(x,y,z) \in \Omega: x = L\}$. The traction is such that the beam deflects in the downward direction and twists about the axis of the beam. 

<img src="results/linear_elastic_problem.png" style="width:400px;">

To describe the large deformation, we introduce few notations:
- $\boldsymbol{X} = (X_1, X_2, X_3) \in \Omega$ denotes the position of a point in the reference configuration (initial configuration) of solid;
- $\boldsymbol{x}(\boldsymbol{X}) = (x_1(\boldsymbol{X}), x_2(\boldsymbol{X}), x_3(\boldsymbol{X})) \in \Omega$ denotes the position of a point $\boldsymbol{X}\in \Omega$ in the current configuration of solid;
- $\boldsymbol{u}(\boldsymbol{X}) = \boldsymbol{x}(\boldsymbol{X}) - \boldsymbol{X}$ denotes the displacement of a reference point $\boldsymbol{X}$;
- $\boldsymbol{F} = \frac{\partial \boldsymbol{x}}{\partial \boldsymbol{X}} = \boldsymbol{I} + \nabla_{\boldsymbol{X}} \boldsymbol{u}(\boldsymbol{X})$ is the deformation gradient, where $\nabla_{\boldsymbol{X}} g = \frac{\partial g}{\partial \boldsymbol{X}} = (\frac{\partial g}{\partial X_1}, \frac{\partial g}{\partial X_2}, \frac{\partial g}{\partial X_3})$ denotes the gradient of a scalar function $g$ with respect to $\boldsymbol{X}$;
- $\boldsymbol{C} = \boldsymbol{F}^T \boldsymbol{F}$ is the right Cauchy-Green strain tensor, where $\boldsymbol{F}^T$ denotes the transpose of a tensor;
- $\boldsymbol{E} = (\boldsymbol{C} - \boldsymbol{I})/2$ is the Lagrangian measure of strain known as the Green strain tensor. We can show that
\begin{equation}\tag{1}
\boldsymbol{E} = \frac{1}{2}\left[\nabla_{\boldsymbol{X}} \boldsymbol{u} + \left(\nabla_{\boldsymbol{X}} \boldsymbol{u} \right)^T + \left(\nabla_{\boldsymbol{X}} \boldsymbol{u} \right)^T \left( \nabla_{\boldsymbol{X}} \boldsymbol{u} \right) \right] = \boldsymbol{e} + \frac{1}{2} \left(\nabla_{\boldsymbol{X}} \boldsymbol{u} \right)^T \left( \nabla_{\boldsymbol{X}} \boldsymbol{u} \right)\,,
\end{equation}
where $\boldsymbol{e} = \frac{1}{2}\left[\nabla_{\boldsymbol{X}} \boldsymbol{u} + \left(\nabla_{\boldsymbol{X}} \boldsymbol{u} \right)^T \right]$ is the linearized strain tensor. Thus, we see that the Green strain tensor $\boldsymbol{E}$ is a nonlinear function of displacement. 
- $\boldsymbol{\sigma}(\boldsymbol{x})$ denotes the Cauchy stress tensor defined in the deformed solid. 
- $\boldsymbol{P}(\boldsymbol{X})$ is the first Piola Kirchhoff stress tensor defined in reference configuration. We have, for $\boldsymbol{x}= \boldsymbol{x}(\boldsymbol{X})$,
\begin{equation}\tag{2}
\boldsymbol{\sigma}(\boldsymbol{x}) = J^{-1} \boldsymbol{P}(\boldsymbol{X}) \boldsymbol{F}(\boldsymbol{X})^T\,.
\end{equation}
In the above, $J = \mathrm{det}(\boldsymbol{F})$ is the determinant of $\boldsymbol{F}$.
- $\boldsymbol{S}(\boldsymbol{X})$ is the second Piola Kirchhoff stress tensor defined in reference configuration. The key property of $\boldsymbol{S}$ is that it is symmetric, and it is often used in place of $\boldsymbol{P}$. The first and second Piola stresses are related as follows: 
\begin{equation}\tag{3}
\boldsymbol{P} = \boldsymbol{F} \boldsymbol{S} \qquad \text{or} \qquad \boldsymbol{S} = \boldsymbol{F}^{-1} \boldsymbol{P},.
\end{equation}
- $\boldsymbol{b}(\boldsymbol{X})$ is the body force per unit volume defined on the reference configuration of solid. If $\bar{\boldsymbol{b}(\boldsymbol{x})}$ denotes the body force on current configuration of solid, we have $\boldsymbol{b}(\boldsymbol{X}) = J \bar{\boldsymbol{b}}(\boldsymbol{x})$.
- $\boldsymbol{t}(\boldsymbol{X})$ is the specified traction (force vector per unit area) at $\boldsymbol{X}$ on some part of the boundary, say $\Gamma_r \in \partial \Omega$. We have, for $\boldsymbol{X} \in \Gamma_r$, $\boldsymbol{t}(\boldsymbol{X}) = \boldsymbol{P}(\boldsymbol{X}) \boldsymbol{n}(\boldsymbol{X})$, where $\boldsymbol{n}(\boldsymbol{X})$ is the unit outward normal at $\boldsymbol{X}$ on the boundary.

We define the three invariants $\mathcal{I}_i(\boldsymbol{C})$ as follows
\begin{equation}\tag{10}
\mathcal{I}_1(\boldsymbol{C}) = \mathrm{tr}(\boldsymbol{C})\,, \quad \mathcal{I}_2(\boldsymbol{C}) = \frac{1}{2}\left[\left(\mathrm{tr}(\boldsymbol{C})\right)^2 - \mathrm{tr}\left( \boldsymbol{C}\right)^2 \right]\,, \quad \mathcal{I}_3(\boldsymbol{C}) = \mathrm{det}(\boldsymbol{C})\,,
\end{equation}
where $\mathrm{tr}$ and $\mathrm{det}$ are trace and determinant operators.

The equilibrium configuration of the body under external traction $\boldsymbol{t}$ on the boundary and body force $\boldsymbol{b}$ is the one where the displacement $\boldsymbol{u}$ satisfies the following boundary value problem:
\begin{align}\tag{4}
-\nabla \cdot \boldsymbol{P} &= \boldsymbol{b} && \qquad \text{in }\;\Omega, \\
 \boldsymbol{F} \boldsymbol{P}^T &= \boldsymbol{P} \boldsymbol{F}^T && \qquad \text{in }\;\Omega, \\
\boldsymbol{u} &= \boldsymbol{0} && \qquad \text{on }\;\Gamma_{l}, \\
\boldsymbol{P}\boldsymbol{n} &= \boldsymbol{t} && \qquad \text{on }\;\Gamma_{r}, \\
\boldsymbol{P}\boldsymbol{n} &= \boldsymbol{0} && \qquad \text{on }\;\partial \Omega - (\Gamma_{r}\cup \Gamma_{l}), \\
\end{align}
where the second equation is the balance of angular momentum, and $\partial \Omega - (\Gamma_{r}\cup \Gamma_{l})$ is the boundary of the domain excluding the left and right surfaces (at $x=0$ and $x=L$).

### Constitutive law

Constitutive law relates the stress, the first Piola Kirchhoff stress tensor $\boldsymbol{P}$, to the strain, the Green Strain tensor $\boldsymbol{F}$. Or we can write the relation between $\boldsymbol{S}$ and $\boldsymbol{E}$, and using the relations between various quantities, we can get the relation between $\boldsymbol{P}$ and $\boldsymbol{F}$. One possibility is to assume material to be linear and isotropic, which results in the following relation:
\begin{equation}\tag{5}
\boldsymbol{S} = \lambda \mathrm{tr}(\boldsymbol{E}) \boldsymbol{I} + 2\mu \boldsymbol{E} = \frac{\lambda}{2} \left[\mathrm{tr}(\boldsymbol{C}) - 3 \right]\boldsymbol{I} + \mu (\boldsymbol{C} - \boldsymbol{I})\,,
\end{equation}
where $(\lambda, \mu)$ are Lam\`e parameters, $\mathrm{tr}(\boldsymbol{\epsilon}) = \epsilon_{ii} = \nabla \cdot \boldsymbol{u}$ the trace of the tensor $\boldsymbol{\epsilon}$, and $\boldsymbol{I} = \delta_{ij}$ the identity tensor in 3-D (in 2-D if this was a 2-D problem). We also used the fact that $\boldsymbol{C} = \boldsymbol{I} + 2\boldsymbol{E}$. This model is also called **neo-Hookean** model and **St Venant-Kirchhoff** model. 

#### Strain energy density
There are other versions of neo Hookean models. In general, we can consider a strain energy density function $W = W(\boldsymbol{X}, \boldsymbol{F})$ that depends on the material point $\boldsymbol{X}$ and deformation gradient $\boldsymbol{F}$. The first Piola stress is given by
\begin{equation}\tag{6}
\boldsymbol{P} = \frac{\partial W}{\partial \boldsymbol{F}}\,,
\end{equation}
and the second Piola stress is given by
\begin{equation}\tag{7}
\boldsymbol{S} = \frac{\partial W}{\partial \boldsymbol{E}} = 2\frac{\partial W}{\partial \boldsymbol{C}}\,.
\end{equation}
Using the above two equations and the relation $\boldsymbol{P} = \boldsymbol{F} \boldsymbol{S}$, we have
\begin{equation}\tag{8}
\boldsymbol{P} = \boldsymbol{F} \frac{\partial W}{\partial \boldsymbol{E}} = 2 \boldsymbol{F} \frac{\partial W}{\partial \boldsymbol{C}}\,.
\end{equation}

**St Venant Kirrchhoff Material Material Model** To recover the relation $\boldsymbol{S} = \lambda \mathrm{tr}(\boldsymbol{E}) \boldsymbol{I} + 2\mu \boldsymbol{E}$, one can take the energy density function as
\begin{equation}\tag{9}
W = \frac{1}{2} \boldsymbol{S} \boldsymbol{:} \boldsymbol{E} = \frac{\lambda}{2} \left[\mathrm{tr}(\boldsymbol{E}) \right]^2 + \mu \mathrm{tr}\left( \boldsymbol{E}^2\right) \,.
\end{equation}
 We can expand the example $W$ above to write it in terms of the invariants $\mathcal{I}_i$ of $\boldsymbol{C}$ as follows
\begin{equation}\tag{11}
W = \mu(\mathcal{I}_1(\boldsymbol{C}) - 3) + \frac{\lambda + 2\mu}{8} \left[\mathcal{I}_1(\boldsymbol{C}) - 3\right]^2 - \frac{\mu}{2} (\mathcal{I}_2(\boldsymbol{C}) - 3) \,.
\end{equation}

**Compressible neo Hookean Material Model** We consider the strain energy that upto first order accounts for energy due to volumetric (purely change in volume keepin the shape same) and isochoric (deformation with no change in volume) deformations:
\begin{equation}\tag{12}
W = \frac{\mu}{2}\left[J^{-2/3}\mathcal{I}_1(\boldsymbol{C}) - 3\right] + \frac{K}{2} (J - 1)^2 \,,
\end{equation}
where recall that $J = \mathrm{det}(\boldsymbol{F})$, and $\mu$ and $K$ are shear and bulk modulii. To compute the stress $\boldsymbol{P}$, we will use the formula $\boldsymbol{P} = \frac{\partial W}{\partial \boldsymbol{F}}$ together with Fenics autodifferentiation feature to compute the derivatives. In the above expression, the first term models the energy contribution due to isochoric (volume-preserved) deformation and the second term models the energy contribution due to volumetric deformation. 


### Variational formulation

Let $V$ be the appropriate function space for the displacement (functions in $V$ are vector-valued functions). Due to homogeneous Dirichlet boundary condition, the trial and test functions belong to the same function space $V$. 

Multiplying the strong form of the problem by test function $\boldsymbol{v}\in V$ and integrating it over the domain $\Omega$ gives:
\begin{equation}\tag{13}
-\int_{\Omega} \left(\nabla \cdot \boldsymbol{P}\right) \cdot \boldsymbol{v} \,d\boldsymbol{X} = \int_{\Omega} \boldsymbol{b} \cdot \boldsymbol{v} \,d\boldsymbol{X} .
\end{equation}
The term on the left can also be written using integration by parts:
\begin{equation}\tag{14}
-\int_{\Omega} \left(\nabla \cdot \boldsymbol{P}\right) \cdot \boldsymbol{v} \,d\boldsymbol{X} = -\int_{\partial \Omega} \left(\boldsymbol{P}\boldsymbol{n}\right)\cdot \boldsymbol{v} \,d\boldsymbol{X} + \int_{\Omega} \boldsymbol{P} \boldsymbol{\colon} \nabla \boldsymbol{v} \,d\boldsymbol{X} = -\int_{\Gamma_{r}} \boldsymbol{t}\cdot \boldsymbol{v} \,d\boldsymbol{X} + \int_{\Omega} \boldsymbol{P} \boldsymbol{\colon} \nabla \boldsymbol{v} \,d\boldsymbol{X}.
\end{equation}
Using the above, the variational problem can be stated as follows:
\begin{equation}\tag{15}
\text{find }\;\boldsymbol{u}\in V \;\text{such that }\qquad \underbrace{\int_{\Omega} \boldsymbol{P} \boldsymbol{\colon} \nabla \boldsymbol{v} \,d\boldsymbol{X}}_{=: a(\boldsymbol{u}; \boldsymbol{v})} = \underbrace{\int_{\Omega} \boldsymbol{b} \cdot \boldsymbol{v} \,d\boldsymbol{X} + \int_{\Gamma_{r}} \boldsymbol{t}\cdot \boldsymbol{v} \,d\boldsymbol{X}}_{=:l(\boldsymbol{v})} \qquad \text{for all }\; \boldsymbol{v}\in V,
\end{equation}
where, $\boldsymbol{P}$ is related to $\boldsymbol{u}$ via the constitutive law, and $a(\cdot; \cdot)$ and $l(\cdot)$ are semilinear (not bilinear! as in the case of linear elasticity) and linear forms. 

### Geometrical, material, and external loading parameters and functions

In what follows, we let $L = 1$ m, $W = H = 0.2$ m. We assume no body force, i.e., $\boldsymbol{b} = \boldsymbol{0}$. The traction loading $\boldsymbol{t} = (t_x, t_y, t_z)$ on the right boundary $\Gamma_{r}$ is given by, for all $\boldsymbol{x} = (x, y, z) \in \Gamma_{r}$:
\begin{equation}\tag{16}
t_x(\boldsymbol{x}) = 0, \quad t_y(\boldsymbol{x}) =  \frac{f_{twist}(z - H/2)}{0.01 + r}, \quad t_z(\boldsymbol{x}) = - f_{bend} - \frac{f_{twist}(y - W/2)}{0.01 + r}, \quad \text{where } r =\sqrt{(y - W/2)^2 + (z - H/2)^2}.
\end{equation}
Here, $f_{twist}$ and $f_{bend}$ are magnitudes of force per unit area controlling the twisting and bending loadings. For the numerics, we fix 
\begin{equation}\tag{17}
f_{twist} = 2\times 10^4\text{ N/m$^2$}, \qquad f_{bend} = 5\times 10^3\text{ N/m$^2$}.
\end{equation}

As for the material properties, we take Young's modulus $E = 10^7$ Pa and Poisson ratio $\nu = 0.3$. These properties are typical of rubber-like materials. Given $(E, \nu)$, the bulk modulus is given by
\begin{equation}\tag{18}
K = \frac{E}{3(1 - 2\nu)}\,.
\end{equation}

### Fenics implementation

We start by loading the relevant packages:

In [1]:
import dolfinx
import numpy as np
import sys
import os

import pyvista


from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

from petsc4py import PETSc

# specific functions from dolfinx modules
from dolfinx import fem, mesh, io, plot, log
from dolfinx.fem import (Constant, dirichletbc, Function, functionspace, Expression )
from dolfinx.fem.petsc import NonlinearProblem
from dolfinx.io import VTXWriter
from dolfinx.nls.petsc import NewtonSolver

# specific functions from ufl modules
import ufl
from ufl import (TestFunctions,TestFunction, TrialFunction, Identity, grad, det, 
                 variable, div, dev, inv, tr, sqrt, conditional ,\
                 gt, dx, inner, derivative, dot, ln, \
                 split, outer, cos, acos, lt, eq, ge, le, exp, diff)

# basix finite elements
import basix
from basix.ufl import element, mixed_element, quadrature_element

# for plotting
import matplotlib.pyplot as plt
plt.close('all')

from datetime import datetime

log.set_log_level(log.LogLevel.WARNING)

Next, we set values of different parameters in the simulation:

In [2]:
# geometry 
omega_L, omega_W, omega_H = 1., 0.2, 0.2

domain = mesh.create_box(MPI.COMM_WORLD, [[0.0,0.0,0.0], [omega_L, omega_W, omega_H]], [20, 4, 4], mesh.CellType.tetrahedron)

# get nodal coordinates in reference configuration
x = ufl.SpatialCoordinate(domain) 

# material
E = Constant(domain, PETSc.ScalarType(1.e+7)) #10 MPa
nu = Constant(domain, PETSc.ScalarType(0.3))
lamda = E*nu/(1+nu)/(1-2*nu)
mu = E/(2*(1+nu))
K = E/(3*(1-2*nu))

# loading
f_twist_max = 2.e+4
f_bend_max = 5.e+3

f_twist = Constant(domain,PETSc.ScalarType(0.))
f_bend = Constant(domain,PETSc.ScalarType(0.))

Define vector finite element function space and scalar finite element space (scalar function space will be used to compute the post-processing quantities such as magnitude of the displacement and von-Mises stress):

In [3]:
# specify order of interpolation
p_order = 2

# create FE element (vector)
Uvec = element("Lagrange", domain.basix_cell(), p_order, shape=(3,))

# FE function space (vector)
Vvec = fem.functionspace(domain, Uvec)

# space for von Mises stress (scalar)
U = element("Lagrange", domain.basix_cell(), p_order)
V = fem.functionspace(domain, U)

We also define the trial and test function, and infer the spatial dimension of the problem:

In [4]:
u = Function(Vvec)
v = TestFunction(Vvec)

# spatial dimension of the problem
d = len(u)
print('d = ', d)

d =  3


Next, define boundaries for boundary conditions:

In [5]:
# define edges/surfaces of beam
def xBot(x):
    return np.isclose(x[0], 0)
def xTop(x):
    return np.isclose(x[0], omega_L)
def yBot(x):
    return np.isclose(x[1], 0)
def yTop(x):
    return np.isclose(x[1], omega_W)
def zBot(x):
    return np.isclose(x[2], 0)
def zTop(x):
    return np.isclose(x[2], omega_H)
    
# mark the sub-domains
boundaries = [(1, xBot),(2,xTop),(3,yBot),(4,yTop),(5,zBot),(6,zTop)]

# build collections of facets on each subdomain and mark them appropriately.
facet_indices, facet_markers = [], [] 
fdim = domain.topology.dim - 1
for (marker, locator) in boundaries:
    facets = mesh.locate_entities(domain, fdim, locator) 
    facet_indices.append(facets) 
    facet_markers.append(np.full_like(facets, marker)) 

# Format the facet indices and markers as required for use in dolfinx.
facet_indices = np.hstack(facet_indices).astype(np.int32)
facet_markers = np.hstack(facet_markers).astype(np.int32)
sorted_facets = np.argsort(facet_indices)
 
# Add these marked facets as "mesh tags" for later use in BCs.
facet_tags = mesh.meshtags(domain, fdim, facet_indices[sorted_facets], facet_markers[sorted_facets])

# create connectivity between the 2D and 3D entities
domain.topology.create_connectivity(domain.topology.dim-1, domain.topology.dim)

Define the surface area measure for integration over the right boundary for the traction boundary condition:

In [6]:
# dx for integration over the domain
# dx = ufl.Measure('dx', domain=domain, metadata={'quadrature_degree': 2})

# ds for integration over the surface
ds = ufl.Measure('ds', domain=domain, subdomain_data=facet_tags, metadata={'quadrature_degree':2})

# normal vector to the surface
n = ufl.FacetNormal(domain)

Implement Dirichlet boundary condition on displacement next:

In [7]:
# Bottom surface displacement degrees of freedom
Btm_dofs_u1 = fem.locate_dofs_topological(Vvec.sub(0), facet_tags.dim, facet_tags.find(1))
Btm_dofs_u2 = fem.locate_dofs_topological(Vvec.sub(1), facet_tags.dim, facet_tags.find(1))
Btm_dofs_u3 = fem.locate_dofs_topological(Vvec.sub(2), facet_tags.dim, facet_tags.find(1))

# Dirichlet BCs into one vector
bcs_0 = dirichletbc(0.0, Btm_dofs_u1, Vvec.sub(0)) 
bcs_1 = dirichletbc(0.0, Btm_dofs_u2, Vvec.sub(1)) 
bcs_2 = dirichletbc(0.0, Btm_dofs_u3, Vvec.sub(2)) 

bcs = [bcs_0, bcs_1, bcs_2]

Next, we define the body force and traction:

In [8]:
class traction_expr():
    def __init__(self, f_twist, f_bend, omega_H, omega_W):
        self.f_twist = f_twist
        self.f_bend = f_bend
        self.omega_H = omega_H
        self.omega_W = omega_W

    def __call__(self, x):
        print('twist', self.f_twist.value, 'bend', self.f_bend.value)
        vals = np.zeros_like(x)
        vals[1,:] = self.f_twist*((x[2, :] - self.omega_H/2)/(np.sqrt(np.pow(x[1, :] - self.omega_W/2, 2) + np.pow(x[2, :] - self.omega_H/2, 2)) + 0.01))
        vals[2,:] = -self.f_twist*((x[1, :] - self.omega_W/2)/(np.sqrt(np.pow(x[1, :] - self.omega_W/2, 2) + np.pow(x[2, :] - self.omega_H/2, 2)) + 0.01)) - self.f_bend
        
        # vals = np.zeros_like(x)
        # vals[1, :] = 1.
        return vals

In [9]:
# body force
b = Constant(domain, PETSc.ScalarType((0., 0., 0.)))#, PETSc.ScalarType)

# traction
# traction_expr = Expression(("0", \
#         "f_twist*((x[2] - omega_H/2)/(sqrt(pow(x[1] - omega_W/2, 2) + pow(x[2] - omega_H/2, 2)) + 0.01))", \
#         "-f_twist*((x[1] - omega_W/2)/(sqrt(pow(x[1] - omega_W/2, 2) + pow(x[2] - omega_H/2, 2)) + 0.01)) - f_bend"), \
#         degree=2, f_twist=f_twist, f_bend=f_bend, omega_H=omega_H, omega_W=omega_W)
t_expr = traction_expr(f_twist, f_bend, omega_H, omega_W)
t_field = Function(Vvec)
t_field.interpolate(t_expr)

twist 0.0 bend 0.0


Finally, we are ready to define the bilinear and linear forms associated with the boundary value problem on displacement:

In [10]:
# Spatial dimension
d = len(u)

# Identity tensor
I = variable(ufl.Identity(d))

# Deformation gradient
F = variable(I + ufl.grad(u))

# Right Cauchy-Green tensor
C = variable(F.T * F)

# Invariants of deformation tensors
Ic = variable(ufl.tr(C))
J = variable(ufl.det(F))

# Stored strain energy density (compressible neo-Hookean model)
W = (mu / 2) * (J**(-2/3)*Ic - 3) + (K/2)*(J - 1)**2

# First Piola-Kirchhoff stress tensor
P = ufl.diff(W, F)

In [11]:
# semilinear form
a = inner(grad(v), P)*dx

# linear form
L = dot(v, b)*dx + dot(v, t_field)*ds

# residual form
R = L - a

Compute post-processing quantities such as von-Mises stress that is defined as
\begin{equation}
\sigma_v = \sqrt{\frac{3}{2}\boldsymbol{\sigma}_{dev}\boldsymbol{\colon}\boldsymbol{\sigma}_{dev}} = \sqrt{\frac{3}{2}\sigma_{dev, ij} \sigma_{dev, ij}}\;, \quad \text{where}\quad  \boldsymbol{\sigma}_{dev} = \text{deviatoric stress} = \boldsymbol{\sigma} - \frac{\mathrm{tr}(\boldsymbol{\sigma})}{3}\boldsymbol{I} 
\end{equation}
where $\sigma = J^{-1} \boldsymbol{P}\boldsymbol{F}^T$.

In [12]:
# cauchy stress from first Piola-Kirchhoff stress
sigma = P*F.T/J

# deviatoric stress
sigma_dev = P - (1./3)*tr(sigma)*Identity(d)  # deviatoric stress
vm = sqrt((3/2)*inner(sigma_dev, sigma_dev))
sigma_vm_expr = Expression(vm, V.element.interpolation_points())
sigma_vm = Function(V, name = 'von Mises stress')
# sigma_vm.interpolate(sigma_vm_expr)

# Compute magnitude of displacement
u_magnitude_expr = Expression(sqrt(dot(u, u)), V.element.interpolation_points())
u_magnitude = Function(V, name = 'magnitude(u)')
# u_magnitude.interpolate(u_magnitude_expr)


# output results
results_folder = "fwd_result/hyperelastic"
os.makedirs(results_folder, exist_ok=True)
filename = results_folder + "/solution"
file_results = io.VTXWriter(domain.comm, filename + ".bp", [u, sigma_vm, u_magnitude], engine="BP4")

def write_sim(t):
    sigma_vm.interpolate(sigma_vm_expr)
    u_magnitude.interpolate(u_magnitude_expr)
    file_results.write(t)

    u_mag_max = u_magnitude.x.array.max()
    u_mag_min = u_magnitude.x.array.min()
    print('min/max u:', u_mag_min, u_mag_max)

ld: warning: duplicate -rpath '/Users/prashant/opt/anaconda3/envs/confenx/lib' ignored
ld: warning: duplicate -rpath '/Users/prashant/opt/anaconda3/envs/confenx/lib' ignored


Since this is a nonlinear problem, we use Fenics in-built nonlinear solver. We start by creating a nonlinear problem and assign values to key solver parameters. 

In [13]:
problem = NonlinearProblem(R, u, bcs)

solver = NewtonSolver(domain.comm, problem)

# Set Newton solver options
solver.atol = 1e-8
solver.rtol = 1e-8
solver.convergence_criterion = "incremental"

ld: warning: duplicate -rpath '/Users/prashant/opt/anaconda3/envs/confenx/lib' ignored
ld: warning: duplicate -rpath '/Users/prashant/opt/anaconda3/envs/confenx/lib' ignored
ld: warning: duplicate -rpath '/Users/prashant/opt/anaconda3/envs/confenx/lib' ignored
ld: warning: duplicate -rpath '/Users/prashant/opt/anaconda3/envs/confenx/lib' ignored


In [ ]:
log.set_log_level(log.LogLevel.INFO)

write_sim(0.0)

for n in range(1, 10):
    f_twist.value = (n/10.) * f_twist_max
    f_bend.value = (n/10.) * f_bend_max
    t_field.interpolate(t_expr)
    num_its, converged = solver.solve(u)
    assert (converged)
    print(f"Time step {n}, Number of iterations {num_its}, Twist Load {f_twist.value}, Bending Load {f_bend.value}")
    write_sim(n)

min/max u: 0.0 0.0
twist 2000.0 bend 500.0
Time step 1, Number of iterations 6, Twist Load 2000.0, Bending Load 500.0
min/max u: 0.0 0.05514760700915548
twist 4000.0 bend 1000.0


### Plotting results in paraview

<img src="results/linear_elastic_results.png" style="width:600px;">
